In [1]:
import sys
sys.path.append("../../")

from cnstats.stats import stats
from macroquant.dataapi.cnstats import *
from datetime import datetime, timedelta

In [2]:
def profit_df(datestr):
    # print(datestr)
    df = cnstats_industry_profit(datestr)
    df2 = cnstats_industry_employment(datestr)
    if len(df2) > 0:
        df = df.join(df2, on='industry', how='inner')
    if len(df) > 0:
        df = df.rename({'利润总额': f'利润总额_{datestr}', '平均用工人数': f'平均用工人数_{datestr}'})
        df = df.with_columns(
            (pl.col(f'利润总额_{datestr}')*10000/pl.col(f'平均用工人数_{datestr}')).alias(f'人均创造利润_{datestr}'),
        )
    return df

In [3]:
start_date, end_date = '202410', '202510'

dates = [start_date, end_date]
date = start_date
df = pl.DataFrame()
for date in dates:
    d = profit_df(date)
    if len(d) > 0:
        if len(df) > 0:
            df = df.join(d, on='industry', how='inner')
        else:
            df = d

df = df.with_columns(
        [
         (pl.col(f'利润总额_{end_date}')).alias(f'利润总额'),
         (pl.col(f'人均创造利润_{end_date}')).alias(f'人均创造利润'),
         ((pl.col(f'利润总额_{end_date}')-pl.col(f'利润总额_{start_date}'))/pl.col(f'利润总额_{start_date}')).alias(f'利润增长率'),
         ((pl.col(f'人均创造利润_{end_date}')-pl.col(f'人均创造利润_{start_date}'))/pl.col(f'人均创造利润_{start_date}')).alias(f'人均创造利润增长率'),
        ]
    )
df.select(pl.col(['industry', '利润总额', '人均创造利润', '利润增长率', '人均创造利润增长率']))


industry,利润总额,人均创造利润,利润增长率,人均创造利润增长率
str,f64,f64,f64,f64
"""采矿业""",7123.3,18.25551,-0.276477,-0.252372
"""煤炭开采和洗选业""",2570.4,10.310469,-0.492176,-0.478528
"""石油和天然气开采业""",2790.6,56.604462,-0.126955,-0.118101
"""黑色金属矿采选业""",381.8,16.819383,-0.221452,-0.183725
"""有色金属矿采选业""",1050.6,44.142857,0.327521,0.299632
…,…,…,…,…
"""金属制品、机械和设备修理业""",172.4,6.223827,-0.022121,-0.120968
"""电力、热力、燃气及水生产和供应业""",7329.3,21.898118,0.108485,0.124713
"""电力、热力生产和供应业""",6264.8,25.855551,0.139655,0.161292


In [4]:
df.select(pl.col(['industry', '利润总额', '人均创造利润', '利润增长率', '人均创造利润增长率'])).write_csv('data/行业利润增长率_过去1年.csv')
